In [1]:
# Setup
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Load env variables
%load_ext dotenv
%dotenv

azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_model_deployment = os.getenv("AZURE_MODEL_DEPLOYMENT")
azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")

# To use API key, set this to False.  
use_entra_id = True

In [2]:
os.system("az login")

0

In [3]:
# Create a client and helper method
if use_entra_id: 
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(exclude_interactive_browser_credential=False), "https://cognitiveservices.azure.com/.default"
    )
    
    client = AzureOpenAI(
        azure_endpoint=azure_oai_endpoint,
        api_version=api_version,
        azure_ad_token_provider=token_provider
    )
else:
    client = AzureOpenAI(
        azure_endpoint=azure_oai_endpoint,
        api_version=api_version,
        api_key=azure_oai_key
    )

def call_o1_model(prompt):    
    response = client.chat.completions.create(
        model=azure_model_deployment,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


In [4]:
# Constraint satisfaction problem - summer scheduling
# Reference for brochure: https://www.schoolcraft.edu/ppl/wp-content/uploads/sites/4/2024/01/20231120_PPL_Kids_On_Campus_Schedule_Summer_2024_05.pdf

prompt = '''
Please plan a weekly schedule for all 3 of my kids for the summer of 2024 (July 8 – August 23).  I would like them to be enrolled in camps most weeks.  Please choose the same 2 weeks for family time where they won’t be in camps.  These 2 weeks don’t have to be consecutive; just pick any two weeks for all 3 kids to be off at the same time.  Here is information about each kid so you can choose the best camps for them.  Tori is 16 (entering grade 12) and loves baking and dance.  Fiona is 12 (entering grade 8) and enjoys reading, writing, and soccer.  Gabe is 10 (entering grade 5) and loves martial arts, superheroes, and video games, especially Minecraft and Roblox.  They all love swimming.  I’m open to exploring other classes they may enjoy, but please make sure they each have at least 2 classes involving these above activities they love.  They should take a different class each week (no duplicates of the same class for the same kid).  Your output should be a matrix of each week of the summer by each kid, and what they are doing that week (either the title of the camp class they would enjoy, or a family time week where they are all off together).  
Here is the summer brochure you can use with the activity listings:

NEW Science in Movies - Fact or Fiction: 
Grades 3 & 4
Do you ever wonder how realistic movies are? Want to learn 
more about the science behind some of the most popular and 
amazing scenes in movie history? If so, then join our science 
in movies camp! Learn how to explore the scientific truth of 
various movies such as Jurassic Park, Star Wars and ET. Use 
the scientific method to test if what you see in the movies are 
based on fact or fiction. 
CES 8560 $249
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939804 F270 Staff
Non-Stop Science Experiments: Grades 3 & 4
All the time is the right time to explore science! Use the scientific 
method to do lots of mind-blowing, kid-tested experiments that 
answer everyday questions. See your world as a ginormous, 
fun-filled playground-laboratory. Roll-up your sleeves, put on your 
safety glasses and get ready to answer the question WHY? 
CES 8533 $249
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939843 F150 Petersen-Price
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939830 F150 C. Janes
Stop Motion Animation: Grades 3 & 4
Lights, camera, LEGO action! Use LEGO® Bricks to tell your story, 
complete with music, special effects and all your favorite LEGO® 
minifigure characters. Plan, script, stage, shoot, and produce your 
own mini-movie using Stop Motion Animation. Working as a team, 
students will use LEGO® components to build the set and props, 
then shoot their movie using a camera. Movie-making software 
will be used to add special effects, titles, credits and more. Each 
camper takes home a copy of their movie.
CES 3782 $275
1 week MTWRF July 29 9 am-3 pm
Sec. 939873 LA140 Staff
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939898 LA140 Staff
9
734-462-4448 | SCHOOLCRAFT.EDU/KOC
GRADES 5 & 6
Harry Potter: Centaurs, 
Dragons & More: Grades 5 & 6
Let your imagination take you on a journey. Sort into your 
houses to discover centaurs, dragons, house elves, owls, 
potions, spells and much more. Get ready for Harry Potter 
trivia and design a broom to play Quidditch. Paint your 
favorite dwelling, draw mythical creatures, and create your 
own wand!
CES 3818 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939897 BTC215 J. Klein
3-Dimensional Art Sculptures: Grades 5 & 6
Experiment with a variety of materials to create one-of-akind art sculptures. Use a variety of art mediums including: 
assemblage, plaster gauze, and more. Design an animal made 
from found objects, tape sculpture, plaster masks, and create 
many more art pieces ready for display. 
CES 3777 $249
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939879 F370 M. Pircer
Zombie Apocalypse 
Survival Training: Grades 5 & 6
Can you survive a zombie apocalypse? Discover the keys to 
survival while learning science, technology, engineering, math 
and of course, having a blast. Learn basic survival skills while 
racing to find a cure and get ready for battle. No zombie camp 
would be complete without the ultimate Nerf gun battle.
CES2 0118 $249
1 week MTWRF July 8 9 am-3 pm
Sec. 939881 BTC300 G. Shumpert
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939870 BTC300 G. Shumpert
ARTS & CULTURE
Add Before & After Care 
to your child’s schedule 
for a full day of fun on campus! 
SEE PAGE 3!
HAVE QUESTIONS? 
Check out our 
KOC Parent Handbook
SCHOOLCRAFT.EDU/KOC
NEW Textile Fashion Illustration & 
Design: Grades 5 & 6
Gain fundamental knowledge of fashion textile and materials. 
Study fabrics, their properties and identification methods. 
Investigate different techniques for creating textiles, 
terminology and create fabric specimens (fabric swatches.) 
Explore different techniques for creating textiles such as dyeing 
and printing. 
CES 3846 $249
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939809 F370 M. Williams
Express Yourself: Grades 5 & 6
Explore your inner self using the arts. Discover multiple 
mediums for self-expression including painting, use of clay, 
acting, sketching, fashion, dance, music, and more. Whether 
it's silliness, joy, frustration etc. learn ways to express yourself 
during this creative arts week.
CES 3836 $249
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939884 BTC215 J. Klein
Anime Drawing & Cartooning: Grades 5 & 6
Learn to create original anime characters and draw your 
personal favorites! Use different styles and techniques for 
creating characters up close and in motion. Start by breaking 
images into whole shapes before drawing them. Create finished 
renderings while exploring tone, shading, dimension, proportion 
and more. 
CES 3819 $249
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939887 LA100 E. Tamulewicz
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939888 LA100 E. Tamulewicz
10
PLEASE BRING A BAG LUNCH TO CAMPS EACH DAY
GRADES 5 & 6
Master the Stage! 
Acting Camp: Grades 5 & 6
If having fun is your favorite thing to do, then this camp is for 
you. Turn your creative side loose as you do script writing, 
improvisation, movement and creative dramatics. Don't worry 
if you have no experience, everyone is welcome. Complete 
your theater experience with your acting debut - an informal 
production for family and friends on the last day.
CES 3700 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939858 LA440 K. Glatz
Kids with Cameras: Grades 5 & 6
Whether you have been taking photos for a while, or are just 
picking up a digital camera for the first time - we want you! 
Open up a whole new world that screams "never bored." Start 
by learning photography basics. Then add some tricks and tips 
and get wild and creative. Fill your world with amazing new 
experiences as you go on photo shoots each day. Learn to use 
Photoshop Elements to improve your photo images. Bring a bag 
lunch and a digital camera to camp each day.
CES 3669 $275
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939831 HS340 Staff
Register your child for the grade level they will be entering in the fall.
Become A Potter!: Grades 5 & 6
Pinch, coil, roll, slam and twist your way into making clay-art! 
Learn studio courtesy and safety, and how to mix and use 
different types of clay. Explore hand-building and firing methods 
and find out lots of ways to decorate and glaze your pieces. 
Enjoy a whole week to plan, make and finish your own ceramic 
treasure. Note: Due to firing and glazing, the majority of your 
projects will not be available until two - three weeks after camp; 
you will receive an email when they are ready. 
CES 3633 $275
1 week MTWRF July 8 9 am-3 pm
Sec. 939855 F410 Staff
1 week MTWRF July 15 9 am-3 pm
Sec. 939836 F410 Staff
Chess - Get Your Game On: Grades 5 & 6
Get your chess game together as you learn to open, trap, 
strategize and do tactical maneuvers that will leave your 
opponents in the dust. Get amped up and take part in 
awesome competitions, tournaments and contests that will 
showcase your game. Go the chess distance!
CES 7336 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939811 LA420 B. Wilson
1 week MTWRF July 22 9 am-3 pm
Sec. 939866 LA420 B. Wilson
Kids Court: Grades 5 & 6
Discover the basics of the legal system, such as the roles 
of judges, lawyers, witnesses, and jurors. Learn how to 
research, prepare, and present a mock trial case with 
your teammates. Practice your public speaking, critical 
thinking, and teamwork skills in a supportive and friendly 
environment. Whether you want to be a lawyer, a judge, or 
just have fun with your friends, the Kids Court Camp will give 
you an unforgettable experience!
CES 7443 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939872 LA100 E. Tamulewicz
11
734-462-4448 | SCHOOLCRAFT.EDU/KOC
GRADES 5 & 6
CAREER EXPLORATION & COMMUNICATION 
Camp Scrubs - Adventures 
in Medicine: Grades 5 & 6
Put on scrubs and enter the world of medicine. Learn about many 
fields in healthcare and what it takes to be a nurse, paramedic, 
test technician or a surgeon. Discover how to take a pulse and 
blood pressure and experience a state-of-the-art simulation lab. 
Learn about the real world of healthcare hands on. 
CES 4278 $279
1 week MTWRF July 15 9 am-3 pm
Sec. 939822 HS310 S. Garza
1 week MTWRF July 22 9 am-3 pm
Sec. 939859 HS310 S. Garza
The Great Debate: Grades 5 & 6
Do you often try to convince your friends that a certain sports 
team, flavor of ice cream or video game is THE best? Your 
friends may disagree with you, and you have to use specific 
examples to help defend your position. Learning how to debate 
means learning how to persuade someone into your way of 
thinking and influence future decision-making. If you want to 
know what it's like to possess the power of persuasion, join us 
for an exciting week of word wars and rebuttals. I challenge you!
CES 7399 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939890 LA360 N. Gordon-Hines
NEW Pokémon® Masters- Designers 
& 3D Makers Unite! Grades 5 & 6
Calling all PokeMasters who want to be the designers of the 
future! Use your Pokémon® imagination and bring your ideas to 
life. Begin by creating your own digital Pokémon®-style custom 
playing card game. Progress onto designing action figures, 
jewelry, and toys in professional-level modeling software. 
Learn how to prepare a model for 3D printing and create a 
design portfolio to showcase your work! No prior experience 
is necessary and 3D designs will be available on a password 
protected Black Rocket website to share with friends and family.
CES2 6481 $339
1 week MTWRF July 29 9 am-3 pm
Sec. 939810 HS340 S. Nakmura
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939812 HS340 S. Nakmura
Register your child for the grade level they will be entering in the fall.
Make It Work! 
Structure & Design: Grades 5 & 6
Have you ever wondered how architects design buildings or how 
engineers develop structures? Roll up your sleeves and get 
ready to make an idea come to life. Design and build your own 
structure. Learn how to use mechanical design and drafting to 
develop prints and design an electrical circuit to integrate within 
a structure to make it work.
CES 8365 $249
1 week MTWRF July 22 9 am-3 pm
Sec. 939868 HS720 M. Batko
The Great Tween Writers Machine: 
Grades 5 & 6
Roar into summer! Get more comfortable as a writer and make a 
ton of new friends. Try your hand at different types of writing like 
autobiographies, circle stories, poetry, even news articles! Get 
into illustrating your works in new and creative ways. Have fun 
hanging with other tweens who love to write as much as you. 
CES 7334 $249
1 week MTWRF July 22 9 am-3 pm
Sec. 939835 LA330 L. Fundkian
Inventors Workshop: Grades 5 & 6
Television. Water Skis. Earmuffs. The popsicle. What do they 
have in common? All were invented by kids! Over 500,000 
kids and teens invent gadgets and games each year. Let your 
imagination run wild and ignite your spirit of inventiveness! As 
a group, take an idea from creation to the patent process to 
completion. Afterward…the sky is the limit!
CES 8543 $279
1 week MTWRF July 8 9 am-3 pm
Sec. 939802 LA140 AccelerateKID
12
PLEASE BRING A BAG LUNCH TO CAMPS EACH DAY
Roblox eSport Game Design: Grades 5-8
Construct and program an eSport game using Roblox Studio 
and the Lua Programming Language. Explore the virtual game 
back end, develop arenas and features, know the toolbox kit 
and build a capture the flag and personal arena for competition. 
Learn game design, computer programming, critical thinking, 
collaboration and communication skills. No game design or 
programming experience required.
CES2 6383 $359
1 week MTWRF July 15 9 am-3 pm
Sec. 939857 JC103 AccelerateKID
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939803 JC103 AccelerateKID
Become a Tech Star! 
Minecraft Modders: Grades 5 & 6
Calling all future programmers and designers! The time is 
now to try this unique gaming experience. Using your favorite 
game, discover the basics of modding and the foundation of 
programming, learning scripting and logic statements, while 
you create your first mod. Experience coding taught through a 
simulated environment inspired by Minecraft. Note: students 
will work in pairs or teams for most of the day. To access your 
project at home you must own a PC/MAC version of Minecraft. 
This course is designed and developed by Black Rocket.
CES2 6289 $339
1 week MTWRF July 8 9 am-3 pm
Sec. 939808 HS210 T. Finklea
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939867 HS210 T. Finklea
Digital Video Storytelling: Grades 5 & 6
In a production team, perfect the art of planning and producing 
digital video media projects. During this hands-on media production 
boot camp, student teams will create commercial advertisements 
and PSAs (public service announcements). Using an Apple iPad as 
a self-contained production studio, student-producers will develop 
scripts and storyboards; explore filming techniques; shoot their 
concept; and fine edit their project. Construct, deliver, and connect: 
our process and workflow strategies will take your digital video 
content and storytelling to the next level.
CES2 6320 $279
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939896 LA110 J. Kaye
GRADES 5 & 6
COMPUTERS/TECHNOLOGY
Add Before & After Care 
to your child’s schedule 
for a full day of fun on campus! 
SEE PAGE 3!
HAVE QUESTIONS? 
Check out our 
KOC Parent Handbook
SCHOOLCRAFT.EDU/KOC
Digital Art - Get Wired: Grades 5 & 6
Go way beyond paint brushes, crayons and colored pencils 
and work with the newest artform, digital art. Create your own 
masterpieces using basic software including Adobe Photoshop 
Elements and online tools. Start by learning design and art 
elements while improving your computer skills and stretching 
your creative horizons. Push your limits as you bring your own 
existing photos to an extreme level of art using collage, digital 
enhancement, layering and more!
CES 3702 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939837 HS210 P. Marabeas
1 week MTWRF July 22 9 am-3 pm
Sec. 939869 HS210 P. Marabeas
Stop Motion Animation: Grades 5 & 6
Use LEGOS® to tell your story, complete with music, special 
effects and all your favorite LEGO® characters. Plan, script, 
stage, shoot, and produce your own mini-movie using Stop 
Motion Animation. Working as a team, students will use LEGO® 
components to build the set and props, then shoot their movie. 
Movie-making software will be used to add special effects, 
titles, credits and more. Each camper takes home a copy of 
their movie.
CES 3835 $275
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939848 LA140 C. Eagling
Web Magic: Grades 5 & 6
Learn tools and tricks to build your own Web page, and the ins 
and outs of posting it on the Web. Try your hand at the basics 
of animation. Discover how to build basic graphics, film clips 
and short animations for your Web page. Create an awesome 
website that will wow your classmates and family! Prerequisite: 
basic computer skills. Bring headphones or ear buds to class.
CES2 6131 $249
1 week MTWRF July 22 9 am-3 pm
Sec. 939840 HS330 G. Jones
13
734-462-4448 | SCHOOLCRAFT.EDU/KOC
GRADES 5 & 6
3-D Video Game & Animation: 
Grades 5 & 6
Go beyond the limits of 2-D game design! Learn the physics 
behind 3-D games, exploring beginner event scripting, level 
design, controlling flow of gameplay and storytelling. Go to the 
next level by creating and animating your own character sprites, 
objects, and backgrounds. Finish with a fully animated character 
to be used in your very own game. Note: no prior experience 
in game design, animation or sketching needed. Students will 
work in pairs or teams for most of the program. Created games 
will be available on a password protected Black Rocket website 
to share with family and friends. Returning students can create 
more advanced projects that build on previous years. Games 
are only compatible on PC computers.
CES2 6339 $339
1 week MTWRF July 15 9 am-3 pm
Sec. 939826 HS330 G. Jones
1 week MTWRF July 29 9 am-3 pm
Sec. 939838 HS210 T. Finklea
Rev Up a Robot: Grades 5 & 6
Robots are sweeping the world. They can be found in 
factories, the military, in space exploration, transportation 
and medicine. The possible uses are mind-boggling and 
never-ending. Learn how to dream up, design, construct and 
program a robot using LEGO® Mindstorms NXT and/or EV3. 
Enjoy the fun of being part of a robotics pit crew that gets a 
robot's motor running.
CES 8462 $279
1 week MTWRF July 8 9 am-3 pm
Sec. 939844 HS600 V. Batko
1 week MTWRF July 15 9 am-3 pm
Sec. 939845 HS600 V. Batko
1 week MTWRF July 22 9 am-3 pm
Sec. 939846 HS600 V. Batko
1 week MTWRF July 29 9 am-3 pm
Sec. 939880 HS600 V. Batko
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939891 HS600 V. Batko
MATH
Mathbusters!: Grades 5 & 6
No challenge is too great for mathbusters! Take math on with 
a vengeance as you dig in and play games, solve riddles and 
complete spine-tingling experiments. Use math skills like 
probability, statistics, story problems and graphing to determine 
if the math problem is confirmed, plausible or busted! 
CES 8458 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939852 LA330 Staff
Add Before & After Care 
to your child’s schedule 
for a full day of fun on campus! 
SEE PAGE 3!
HAVE QUESTIONS? 
Check out our 
KOC Parent Handbook
SCHOOLCRAFT.EDU/KOC
SCIENCE
Geology Rocks!: Grades 5 & 6
Dig into the fascinating world of Geology. Explore topics such as 
minerals, rocks and fossils, plate tectonics, topographic maps 
and more. Investigate geologic hazards such as landslides and 
volcanoes. Have fun, get your hands dirty and become a rock 
hound in this hands-on camp!
CES 8544 $249
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939865 F350 Staff
ZooCrew - Take A Walk 
on the Wild Side: Grades 5 & 6
Get an insider's look into the amazing world of animals and 
their various ecosystems. Using the scientific method you'll do 
outdoor studies, pond collections, fun labs and cool projects. 
Enjoy a visit from a variety of live animals. Note: all activities 
with animals will be closely supervised. 
CES 8514 $279
1 week MTWRF July 15 9 am-3 pm
Sec. 939847 HS740 D. Craig
14
PLEASE BRING A BAG LUNCH TO CAMPS EACH DAY
ARTS & CULTURE
Bakers Academy: Grades 7, 8 & 9
If baking is your passion, take five days and join the Baking 
Academy. Raise your baking skill level no matter where your 
skills are currently. Begin with basic sanitation and baking 
skills. Once mastered, you'll prepare a scrumptious variety of 
pies, cookies, breads, muffins, coffee cake, French croissants 
and brioche! Take home a new creation daily impressing 
your family with your new skills. No experience necessary. 
A chef coat will be provided along with a drink and dessert. 
Requirements: bring a tool kit with the following: a paring and 
chef knife, peeler, heat proof spatula and measuring spoons. 
Wear long pants and comfortable, closed-toe shoes; long hair 
should be tied back. 
KOC Culinary camps are very intensive.
CES 2510 $349
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939249 VT645 J. Gabriel, CMC
Vegetarian Cooking: Grades 7, 8 & 9
Even if you do not eat vegetarian, meatless meals, snacks, 
desserts, and side dishes are great options for something 
different. Enjoy creating enticing vegetarian meals like 
meat-free pizza, burgers, soups and more, using fresh, 
high-quality ingredients. Dive into a world of new fruits and 
vegetables expanding your palette, discovering firsthand 
how satisfying and flavorful vegetarian dishes can be. Bring 
a bag lunch to camp each day. A chef coat will be provided. 
Requirements: bring a tool kit with the following: a paring 
and chef knife, peeler, heatproof spatula and measuring 
spoons. Wear long pants and comfortable, closed-toe shoes; 
long hair should be tied back. 
KOC Culinary camps are very intensive.
CES 2586 $349
1 week MTWRF July 29 9 am-3 pm
Sec. 939256 VT645 A. Sayes, CMC
The Potter's Studio: Grades 7, 8 & 9
Let your creativity explode! Learn studio courtesy and safety, 
and then how to design and make ceramic pieces using the 
pinch, coil and slab techniques. When you're ready, try your 
hand at working on the potter's wheel. Whether you've ever tried 
ceramics or not, the Potter's Studio is the place for you to get 
into this awesome means of expression and show the world 
what you're all about! Note: Due to firing and glazing the majority 
of your projects will not be available until two-three weeks after 
camp; you will receive an email when they are ready. 
CES 3635 $279
1 week MTWRF July 22 9 am-3 pm
Sec. 939234 F410 Staff
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939267 F410 Staff
GRADES 7, 8 & 9
Register your child for the grade level they will be entering in the fall.
15
734-462-4448 | SCHOOLCRAFT.EDU/KOC
The Art Cafe: Grades 7, 8 & 9
Are you passionate about art and want to escape to a place 
where you can create more? Come to the Art Cafe. Learn by 
doing as you try your hand at different styles and techniques 
using a variety of mediums such as painting with watercolors 
and acrylics, and print techniques using ink. Enjoy the laid-back, 
comfortable atmosphere where you can create, create, create!
CES 3602 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939277 F470 M. Pircer
1 week MTWRF July 29 9 am-3 pm
Sec. 939272 F470 M. Pircer
NEW Song Studio: Grades 7-9
Do you love music and want to learn how to write your own songs? 
Join us for an exciting camp where you will explore the basics of 
melody, harmony, lyrics, and rhythm. You will also learn how to use 
some of the best free songwriting tools online. Complete at least 
one original song that you can be proud of. Unleash your musical 
potential and have fun along the way. No experience needed. 
CES 3847 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939241 JC127 B. Chenoweth
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939242 JC127 B. Chenoweth
Anime & Manga Drawing: Grades 7, 8 & 9
Draw your favorite characters and create new anime and 
manga style characters. Use different styles and techniques 
for creating characters up close and in motion. Create finished 
renderings while exploring tone, shading, dimension, proportion 
and more. Students will be encouraged to share their drawings.
CES 3820 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939230 LA100 E. Tamulewicz
1 week MTWRF July 22 9 am-3 pm
Sec. 939227 LA100 E. Tamulewicz
Curtain Call: Honing Your 
Acting Skills: Grades 7-9
Act in an inspiring and friendly environment. Discover how 
to tell a story with ensemble building games, improv, script 
analysis, monologues, scene work, rehearsal and performance 
tools. Collaborate with fellow actors and work towards a final 
production to be performed for family and friends. For novice 
and experienced performers.
CES 3771 $249
1 week MTWRF July 8 9 am-3 pm
Sec. 939243 LA440 K. Glatz
Chess - Capture the King: Grades 7, 8 & 9
Become lord of the board and take your chess game to the next 
level. Analyze, strategize and maneuver your way into taking 
down your opponents while learning popular chess opening 
moves, how to adapt to your opponent's moves, and the art 
of setting tactical traps. Through lecture and practice games, 
rev up your game and then advance to competitions and 
tournaments. Note: Students must have a basic knowledge of 
chess to participate. 
CES 7425 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939233 LA420 B. Wilson
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939239 LA420 B. Wilson
Digital Photography - Let Loose 
& Experience It!: Grades 7, 8 & 9
Digital photography is hot - get in on the fun! Discover exactly 
what you'll need to know to capture those parties, school 
events, sleepovers and family vacations. Every important 
question gets an easy-to-understand answer: What is resolution 
and why does it matter? How can I make computers, scanners, 
and printers work with my camera? Start by learning camera 
basics. Go on daily photo shoots. Then dive into Photoshop 
Elements and enhance your photos. Bring a digital camera to 
camp each day.
CES 3680 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939260 HS330 Staff
GRADES 7, 8 & 9
Register your child for the grade level they will be entering in the fall.
16
PLEASE BRING A BAG LUNCH TO CAMPS EACH DAY
CAREER EXPLORATION COMMUNICATION 
The Great-er Debate: Grades 7, 8 & 9
Should homework be banned? Are sports stars paid too much? 
Is social media helping or hurting our society? You might be 
brooding over these issues, but don't know how to express your 
opinion to others. Gain the skills to confidently argue either 
side of the debate. Work with a team of like-minded students to 
strategize, mobilize and mesmerize - wow your family and friends 
with word wizardry. Who knows, by the end of the week, you might 
even be able to convince your parents to raise your allowance! 
CES 7412 $249
1 week MTWRF July 22 9 am-3 pm
Sec. 939284 LA360 N. Gordon-Hines
GRADES 7, 8 & 9
3-D Printing & Design: Grades 7, 8 & 9
How cool would it be to turn an idea into reality? Following 
the same process as an industrial designer - read blueprints, 
draw and sketch an image, add dimensions; and turn an idea 
into a 3-D model using Tinker cad. Then bring the idea to life 
with a 3-D printer. Come with your imagination and ideas and 
create a real life prototype! Class meets at the Manufacturing 
& Engineering Center, 13001 Merriman Rd., Livonia, MI 48150. 
Due to the printing time, projects will be available two to 
three weeks after the camp ends. An e-mail will be sent 
when they are ready for pick up.
CES2 6341 $279
1 week MTWRF July 8 8:30 am-2:30 pm
Sec. 939257 MEC120 A. Williams
1 week MTWRF July 15 8:30 am-2:30 pm
Sec. 939258 MEC120 A. WIlliams
Camp Scrubs - Adventures 
in Medicine: Grades 7, 8 & 9
Put on scrubs and enter the world of medicine. Learn about many 
fields in healthcare and what it takes to be a nurse, paramedic, 
test technician or a surgeon. Discover how to take a pulse 
and blood pressure, start an I.V. and become first aid and CPR 
certified. Learn about the real world of healthcare hands on. 
CES 4290 $279
1 week MTWRF July 8 9 am-3 pm
Sec. 939283 HS310 S. Garza
1 week MTWRF July 29 9 am-3 pm
Sec. 939280 HS310 S. Garza
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939254 HS310 S. Garza
Make It Work! Structure 
& Design: Grades 7, 8 & 9
Have you ever wondered how architects design buildings 
or how engineers develop structures? Roll up your sleeves 
and get ready to make an idea come to life. Design and build 
your own structure. Learn how to use mechanical design and 
drafting to develop prints and design an electrical circuit to 
integrate within a structure to make it work.
CES 8470 $249
1 week MTWRF July 29 9 am-3 pm
Sec. 939286 HS720 Staff
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939275 HS720 Staff
Ignite Your Firefighter Power: Grades 7-12
Take advantage of this once-in-a-lifetime opportunity to become a 
real firefighter for an entire week! Work in a fire training facility with 
all the tools, equipment, gear, simulators and vehicles that our 
Fire Academy students use on a daily basis. Consider it a sneak 
peek into a future career choice, or just gain a greater appreciation 
for what it means to "answer the call." Either way, it is sure to 
"spark" your interest! Note: Wear flat, closed-toe/heeled shoes (no 
sandals). Camp activities take place both at Schoolcraft's Livonia 
Campus and Fire Training Facility. Transportation to and from the Fire 
Training Facility is provided. Bring a bag lunch to camp each day.
CES2 0005 $299
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939231 LA230 Staff
Teen Writers Inc.: Grades 7, 8 & 9
Spend time with other teens who love to write as much as you 
do. Define and fine tune your own writing process while enjoying 
free writing time, peer editing and some computer illustration 
opportunities. Write both narratives and poetry focusing on 
character, plot, theme and scene development. Get down to 
business and become a writer extraordinaire!
CES 7335 $249
1 week MTWRF July 22 9 am-3 pm
Sec. 939269 BTC300 C. Ridley
17
734-462-4448 | SCHOOLCRAFT.EDU/KOC
GRADES 7, 8 & 9
COMPUTERS/TECHNOLOGY
3-D Game Design with Unity: Grades 7, 8 & 9
Are you ready to take your game design skills to another 
level? With Unity, an industry-grade design software, aspiring 
game designers will learn level editing, 3-D modeling, 
impactful gameplay creation, as well as how to utilize scripts 
and variables. Student-created games will be available on a 
password protected Black Rocket website to share with friends 
and family. Students will work in pairs or teams for most of 
the program. Returning students can create more advanced 
projects that build on previous years.
CES2 6379 $339
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939268 HS330 G. Jones
Be the Next Social Media Sensation: 
Grades 7, 8 & 9
Perfect the art of planning and producing digital video media 
projects for social media, including YouTube®. Learn to create 
videos that your audience will relate to and produce high-quality 
digital video content. Understand the do's and don't of several 
social media platforms while developing your media personality. 
Use your cell phone and an Apple iPad as a self-contained 
production studio. Student producers will develop scripts, 
explore filming techniques, shoot their concept, and fine edit 
their projects. Walk away with the skills needed to be the next 
YouTube® star.
CES2 6378 $279
1 week MTWRF July 15 9 am-3 pm
Sec. 939246 LA110 J. Kaye
Add Before & After Care 
to your child’s schedule 
for a full day of fun on campus! 
SEE PAGE 3!
Creating Code with Python: Grades 7, 8 & 9
Are you ready to move beyond "drag and drop designs?" 
Learning to code is the key to opening your programming 
potential. Coding is used for everything from games, animation, 
websites and more. Learn Python, a general purpose coding 
language that allows you to express contents with just a few 
lines of code! Requirement: Students should be comfortable 
using a computer, navigating the Internet, and managing files. 
Some experience with coding including programs such as 
Scratch and Alice is helpful but not required.
CES2 6263 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939273 JC113 Staff
Minecraft Modders: Grades 7, 8 & 9
Interested in programming and designing? Try this unique 
gaming experience. Using your favorite game, learn the basics 
of modding and the fundamentals of programming, learning 
scripting and logic statements while you create your first mod. 
Experience coding taught through a simulated environment 
inspired by Minecraft. Note: Students will work in pairs or 
teams for most of the day. To access your project at home 
you must own a PC/MAC version of Minecraft. This course is 
designed and developed by Black Rocket.
CES2 6290 $339
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939235 HS330 S. Nakamura
JavaScript: Grades 7, 8 & 9
Explore programming as a career path with JavaScript. Through 
practical projects JavaScript is used to code solutions for real 
world programming needs. Boost your critical thinking and 
problem-solving skills.
CES2 6471 $319
1 week MTWRF July 22 9 am-3 pm
Sec. 939262 JC113 Staff
Web Design: Grades 7, 8 & 9
Learn how to design your own Web page and the ins and 
outs of posting it on the Web. Try your hand at the basics of 
animation. Discover how to build basic graphics, film clips 
and short animations - the building blocks for your Web page. 
Impress everyone with your newly created Web page! Bring 
headphones or ear buds to class.
CES2 6132 $249
1 week MTWRF July 8 9 am-3 pm
Sec. 939264 HS330 G. Jones
18
PLEASE BRING A BAG LUNCH TO CAMPS EACH DAY
GRADES 7, 8 & 9
SCIENCE
Robotics for Teens: Grades 7, 8 & 9
Robots are used in everything from building cars to space 
travel. Discover the principles of real robotic engineering as 
you help dream up, design, build and program a robot using 
LEGO® Mindstorms NXT and/or EV3. Learn about electrical 
and mechanical engineering as you perform different robot 
experiments. Examine the history of robotics and find out how 
robots compete. Have fun and experience being part of a teen 
robotics team.
CES 8465 $279
1 week MTWRF July 8 9 am-3 pm
Sec. 939253 HS830 P. Marabeas
1 week MTWRF July 15 9 am-3 pm
Sec. 939265 HS830 T. Finklea
1 week MTWRF July 22 9 am-3 pm
Sec. 939289 HS830 T. Finklea
1 week MTWRF July 29 9 am-3 pm
Sec. 939290 HS830 P. Marabeas
1 week MTWRF Aug 5 9 am-3 pm
Sec. 939292 HS830 P. Marabeas
Register your child for the grade level they will be entering in the fall.
HAVE QUESTIONS? 
Check out our 
KOC Parent Handbook
SCHOOLCRAFT.EDU/KOC
Myth Busters Science: Grades 7,8 & 9
Unravel the mysteries of modern science as a member of a 
scientific investigation team. You'll help pull apart and uncover 
the truth like on TV's popular show MythBusters. Your team 
will choose a science myth to test. You'll design investigations, 
choose experiments, test theories and do data analysis. On the 
final day your team will lay it all out - confirmed or busted!
CES 8466 $249
1 week MTWRF July 15 9 am-3 pm
Sec. 939237 F270 Staff
Genetics & You!: Grades 7, 8 & 9
What is more interesting than you? Understand the fundamentals 
of genetics including the role of DNA in your body. Participate in 
hands-on activities such as building model systems using candy 
and extracting strawberry DNA. Engage in lab activities to explore 
the molecular and organismal aspects of inheritance. Discover 
how genetics influences your appearance and health, and how the 
environment can shape it as well.
CES 8558 $279
1 week MTWRF July 22 9 am-3 pm
Sec. 939232 F270 Staff
Bio/Chem - One World: Grades 7, 8 & 9
The world around you is a humungous science lab waiting to 
happen. Look closely at everyday things and discover how their 
chemical and biological makeup defines them. Through information 
sharing and exciting labs you'll see science in a whole new light. 
Don't worry if you are not really experienced with science, the class 
is flexible and may be adjusted to suit any range of abilities.
CES 8468 $249
1 week MTWRF July 8 9 am-3 pm
Sec. 939236 F270 M. Gradinscak
1 week MTWRF Aug 12 9 am-3 pm
Sec. 939291 F270 M. Gradinscak
Zoology That Rocks!: Grades 7, 8 & 9
Dive into the science that covers everything animal, from 
microscopic investigation to comparative studies of animal 
anatomy, physiology and genetics, behavior and environmental 
concerns. The focus is on hands-on fun, wildlife, and the 
outdoors through participation. Note: All activities with animals 
will be closely supervised.
CES 8518 $279
1 week MTWRF July 22 9 am-3 pm
Sec. 939238 HS740 D. Craig
19
734-462-4448 | SCHOOLCRAFT.EDU/KOC
NEW Bakers Workshop: Grades 10-12
Do you love baking and want to learn new skills and recipes? 
Then join our baking camp! Our baking camp is a fun and 
educational program that will teach you how to bake various 
types of cakes, cookies, pies, breads, and more. You will learn 
from professional chefs who will guide you through the basics 
of baking, such as measuring, mixing and kneading. Each day 
you will bring home delicious items to share with your family. 
A chef coat will be provided along with a drink and dessert. 
Requirements: bring a tool kit with the following: a paring 
and chef knife, peeler, heat proof spatula and measuring 
spoons. Wear long pants and comfortable, closed-toe shoes; 
long hair should be tied back. Note: KOC Culinary camps 
are very intensive.
CES 2725 $349
1 week MTWRF July 22 9 am-3 pm
Sec. 939240 VT630 J. Gabriel, CMC
NEW Microscopy - Exploring the Invisible 
World: Grades 9-12
Visit a mysterious invisible world using unparalleled 
microscopic technologies. In Schoolcraft’s renowned Imaging 
Lab, which houses the college’s state-of-the-art Scanning 
Electron Microscope learn how to operate a compound light 
microscope, stereo dissecting microscope and the college's 
very own Scanning Electron Microscope! Capture and save your 
discoveries using digital cameras and even print a poster of 
your favorite image to take home.
CES 8564 $319
1 week MTWRF July 29 9 am- 3 pm
Sec. 939228 BTC175 M. Gury
Ceramic Arts: Sophomore - Senior
In a professional ceramics studio explore the numerous 
possibilities clay has to offer such as: hand building, sculpting, 
decorating, glazing and wheel throwing. Whether you have tried 
ceramics or not, this is the place for you to experience this 
awesome art form and show the world what you're all about! 
Note: Due to firing and glazing the majority of your projects 
will not be available until two - three weeks after camp; you will 
receive an email when they are ready.
CES 3753 $279
1 week MTWRF July 29 9 am-3 pm
Sec. 939248 F410 Staff
HAVE QUESTIONS? 
Check out our 
KOC Parent Handbook
SCHOOLCRAFT.EDU/KOC
Fashion Design & Illustration: Grades 9-12
Define your design identity, art form and brand identity. Learn 
the basics of fashion illustration and design including drawing 
techniques of the human figure. Explore different methods such 
as using pencil, markers, watercolors and paint. Create a mood 
board, develop a theme for your collection, prepare a portfolio 
and more!
CES 3839 $249
1 week MTWRF July 8 9 am-3 pm
Sec. 939245 F370 M. Williams
Welding Techniques & 
Unique Art: Sophomore - Senior
Learn the skills and process of welding. Begin by learning 
the basic safety skills and knowledge of metal welding and 
fabrication. Understand the basics of MIG, TIG, Stick & Oxyacetylene welding methods, as well as other fabrication 
techniques. Once you have some skills developed, showcase 
them by creating a metal art sculpture. For safety reasons wear 
closed-toe shoes, a 100 percent cotton or natural fiber longsleeved shirt and long pants. Please, no synthetic clothing. 
Additional safety equipment will be provided. 
CES 3687 $279
1 week MTWRF Aug 5 8:30 am-2:30 pm
Sec. 939229 MEC130 D. Stuart
'''
result = call_o1_model(prompt)
print(result)

Below is one possible schedule covering all seven weeks (July 8 – August 23), with two shared “family time” weeks at the end. Each row is a week; each column shows Tori’s, Fiona’s, and Gabe’s activities. Weeks 6 and 7 are reserved for family time for all three kids together. No classes repeat for the same child, and each child gets at least two camps that align with their favorite interests (baking/dance for Tori; reading/writing/soccer for Fiona; martial arts/superheroes/Minecraft/Roblox for Gabe).

 ┌────────────────────────┬─────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────┐
 │         Week           │ Tori (16, entering Gr. 12)                            │ Fiona (12, entering Gr. 8)                                     │ Gabe (10, entering Gr. 5)                                             │
 ├────────────────────────┼────────────────────────────

In [ ]:
# Checking two writings for differences and composing - Azure Content Safety API example

# Here are some links to help with analysis.
# To see what's actually new: https://learn.microsoft.com/en-us/azure/ai-services/content-safety/whats-new
# To see APIs by version: https://learn.microsoft.com/en-us/rest/api/contentsafety/operation-groups?view=rest-contentsafety-2023-10-01
# Old version: https://learn.microsoft.com/en-us/rest/api/contentsafety/text-operations/analyze-text?view=rest-contentsafety-2023-04-30-preview&tabs=HTTP
# New version: https://learn.microsoft.com/en-us/rest/api/contentsafety/text-operations/analyze-text?view=rest-contentsafety-2024-02-15-preview&tabs=HTTP

prompt = '''
Using the provided documentation below, summarize the key changes between API versions.  Then craft a blog post announcing the changes for audience of software developers.

# Version 2023-04-30-preview  
Text Operations - Analyze Text
Reference
Service:
Azure AI Services
API Version:
2023-04-30-preview
Analyze Text
A sync API for harmful content analysis for text. Currently, we support four categories: Hate, SelfHarm, Sexual, Violence.

HTTP

Copy

Try It
POST {endpoint}/contentsafety/text:analyze?api-version=2023-04-30-preview
URI Parameters
Name	In	Required	Type	Description
endpoint	path	True	
string

Supported Cognitive Services endpoints (protocol and hostname, for example: https://.cognitiveservices.azure.com).

api-version	query	True	
string

The API version to use for this operation.

Request Header
Name	Required	Type	Description
Ocp-Apim-Subscription-Key	True	
string

Request Body
Name	Required	Type	Description
text	True	
string

The text needs to be scanned. We support at most 1000 characters (unicode code points) in text of one request.

blocklistNames		
string[]

The names of blocklists.

breakByBlocklists		
boolean

When set to true, further analyses of harmful content will not be performed in cases where blocklists are hit. When set to false, all analyses of harmful content will be performed, whether or not blocklists are hit.

categories		
TextCategory[]

The categories will be analyzed. If not assigned, a default set of the categories' analysis results will be returned.

Responses
Name	Type	Description
200 OK	
AnalyzeTextResult

The request has succeeded.

Other Status Codes	
Azure.Core.Foundations.ErrorResponse

An unexpected error response.

Headers

x-ms-error-code: string

Security
Ocp-Apim-Subscription-Key
Type: apiKey
In: header
Examples
Analyze Text
Sample request
HTTP
HTTP

Copy
POST {endpoint}/contentsafety/text:analyze?api-version=2023-04-30-preview

{
  "text": "This is text example"
}

Sample response
Status code:
200
JSON

Copy
{
  "blocklistsMatchResults": [],
  "hateResult": {
    "category": "Hate",
    "severity": 0
  },
  "selfHarmResult": {
    "category": "SelfHarm",
    "severity": 0
  },
  "sexualResult": {
    "category": "Sexual",
    "severity": 0
  },
  "violenceResult": {
    "category": "Violence",
    "severity": 0
  }
}
Definitions
Name	Description
AnalyzeTextOptions	
The analysis request of the text.

AnalyzeTextResult	
The analysis response of the text

Azure.Core.Foundations.Error	
The error object.

Azure.Core.Foundations.ErrorResponse	
A response containing error details.

Azure.Core.Foundations.InnerError	
An object containing more specific information about the error. As per Microsoft One API guidelines - https://github.com/Microsoft/api-guidelines/blob/vNext/Guidelines.md#7102-error-condition-responses.

TextAnalyzeSeverityResult	
Text analysis result.

TextBlocklistMatchResult	
The result of blocklist match.

TextCategory	
Text analyze category

AnalyzeTextOptions
The analysis request of the text.

Name	Type	Description
blocklistNames	
string[]

The names of blocklists.

breakByBlocklists	
boolean

When set to true, further analyses of harmful content will not be performed in cases where blocklists are hit. When set to false, all analyses of harmful content will be performed, whether or not blocklists are hit.

categories	
TextCategory[]

The categories will be analyzed. If not assigned, a default set of the categories' analysis results will be returned.

text	
string

The text needs to be scanned. We support at most 1000 characters (unicode code points) in text of one request.

AnalyzeTextResult
The analysis response of the text

Name	Type	Description
blocklistsMatchResults	
TextBlocklistMatchResult[]

The details of blocklist match.

hateResult	
TextAnalyzeSeverityResult

Analysis result for Hate category.

selfHarmResult	
TextAnalyzeSeverityResult

Analysis result for SelfHarm category.

sexualResult	
TextAnalyzeSeverityResult

Analysis result for Sexual category.

violenceResult	
TextAnalyzeSeverityResult

Analysis result for Violence category.

Azure.Core.Foundations.Error
The error object.

Name	Type	Description
code	
string

One of a server-defined set of error codes.

details	
Azure.Core.Foundations.Error[]

An array of details about specific errors that led to this reported error.

innererror	
Azure.Core.Foundations.InnerError

An object containing more specific information than the current object about the error.

message	
string

A human-readable representation of the error.

target	
string

The target of the error.

Azure.Core.Foundations.ErrorResponse
A response containing error details.

Name	Type	Description
error	
Azure.Core.Foundations.Error

The error object.

Azure.Core.Foundations.InnerError
An object containing more specific information about the error. As per Microsoft One API guidelines - https://github.com/Microsoft/api-guidelines/blob/vNext/Guidelines.md#7102-error-condition-responses.

Name	Type	Description
code	
string

One of a server-defined set of error codes.

innererror	
Azure.Core.Foundations.InnerError

Inner error.

TextAnalyzeSeverityResult
Text analysis result.

Name	Type	Description
category	
TextCategory

The text category.

severity	
integer

The higher the severity of input content, the larger this value is. The values could be: 0,2,4,6.

TextBlocklistMatchResult
The result of blocklist match.

Name	Type	Description
blockItemId	
string

The id of matched item.

blockItemText	
string

The content of matched item.

blocklistName	
string

The name of matched blocklist.

length	
integer

The length of matched text in original input.

offset	
integer

The character offset of matched text in original input.

TextCategory
Text analyze category

Name	Type	Description
Hate	
string

SelfHarm	
string

Sexual	
string

Violence	
string

# -------------------------------------
# Version 2024-02-15-preview

Text Operations - Analyze Text
Reference
Service:
Azure AI Services
API Version:
2024-02-15-preview
Analyze Text
A synchronous API for the analysis of potentially harmful text content. Currently, it supports four categories: Hate, SelfHarm, Sexual, and Violence.

HTTP

Copy

Try It
POST {endpoint}/contentsafety/text:analyze?api-version=2024-02-15-preview
URI Parameters
Name	In	Required	Type	Description
endpoint	path	True	
string

Supported Cognitive Services endpoints (protocol and hostname, for example: https://.cognitiveservices.azure.com).

api-version	query	True	
string

The API version to use for this operation.

Request Body
Name	Required	Type	Description
text	True	
string

The text needs to be analyzed. We support a maximum of 10k Unicode characters (Unicode code points) in the text of one request.

blocklistNames		
string[]

The names of blocklists.

categories		
TextCategory[]

The categories will be analyzed. If they are not assigned, a default set of analysis results for the categories will be returned.

haltOnBlocklistHit		
boolean

When set to true, further analyses of harmful content will not be performed in cases where blocklists are hit. When set to false, all analyses of harmful content will be performed, whether or not blocklists are hit.

outputType		
AnalyzeTextOutputType

This refers to the type of text analysis output. If no value is assigned, the default value will be "FourSeverityLevels".

Responses
Name	Type	Description
200 OK	
AnalyzeTextResult

The request has succeeded.

Other Status Codes	
Azure.Core.Foundations.ErrorResponse

An unexpected error response.

Headers

x-ms-error-code: string

Security
Ocp-Apim-Subscription-Key
Type: apiKey
In: header
OAuth2Auth
Type: oauth2
Flow: application
Token URL: https://login.microsoftonline.com/common/oauth2/v2.0/token
Scopes
Name	Description
https://cognitiveservices.azure.com/.default	
Examples
Analyze Text
Sample request
HTTP
HTTP

Copy
POST {endpoint}/contentsafety/text:analyze?api-version=2024-02-15-preview

{
  "text": "This is text example"
}

Sample response
Status code:
200
JSON

Copy
{
  "blocklistsMatch": [],
  "categoriesAnalysis": [
    {
      "category": "Hate",
      "severity": 0
    },
    {
      "category": "SelfHarm",
      "severity": 0
    },
    {
      "category": "Sexual",
      "severity": 0
    },
    {
      "category": "Violence",
      "severity": 0
    }
  ]
}
Definitions
Name	Description
AnalyzeTextOptions	
The text analysis request.

AnalyzeTextOutputType	
This refers to the type of text analysis output. If no value is assigned, the default value will be "FourSeverityLevels".

AnalyzeTextResult	
The text analysis response.

Azure.Core.Foundations.Error	
The error object.

Azure.Core.Foundations.ErrorResponse	
A response containing error details.

Azure.Core.Foundations.InnerError	
An object containing more specific information about the error. As per Microsoft One API guidelines - https://github.com/Microsoft/api-guidelines/blob/vNext/Guidelines.md#7102-error-condition-responses.

TextBlocklistMatch	
The result of blocklist match.

TextCategoriesAnalysis	
Text analysis result.

TextCategory	
Text analyze category.

AnalyzeTextOptions
The text analysis request.

Name	Type	Default value	Description
blocklistNames	
string[]

The names of blocklists.

categories	
TextCategory[]

The categories will be analyzed. If they are not assigned, a default set of analysis results for the categories will be returned.

haltOnBlocklistHit	
boolean

When set to true, further analyses of harmful content will not be performed in cases where blocklists are hit. When set to false, all analyses of harmful content will be performed, whether or not blocklists are hit.

outputType	
AnalyzeTextOutputType

FourSeverityLevels	
This refers to the type of text analysis output. If no value is assigned, the default value will be "FourSeverityLevels".

text	
string

The text needs to be analyzed. We support a maximum of 10k Unicode characters (Unicode code points) in the text of one request.

AnalyzeTextOutputType
This refers to the type of text analysis output. If no value is assigned, the default value will be "FourSeverityLevels".

Name	Type	Description
EightSeverityLevels	
string

Output severities in eight levels, the value could be 0,1,2,3,4,5,6,7.

FourSeverityLevels	
string

Output severities in four levels, the value could be 0,2,4,6.

AnalyzeTextResult
The text analysis response.

Name	Type	Description
blocklistsMatch	
TextBlocklistMatch[]

The blocklist match details.

categoriesAnalysis	
TextCategoriesAnalysis[]

Analysis result for categories.

Azure.Core.Foundations.Error
The error object.

Name	Type	Description
code	
string

One of a server-defined set of error codes.

details	
Azure.Core.Foundations.Error[]

An array of details about specific errors that led to this reported error.

innererror	
Azure.Core.Foundations.InnerError

An object containing more specific information than the current object about the error.

message	
string

A human-readable representation of the error.

target	
string

The target of the error.

Azure.Core.Foundations.ErrorResponse
A response containing error details.

Name	Type	Description
error	
Azure.Core.Foundations.Error

The error object.

Azure.Core.Foundations.InnerError
An object containing more specific information about the error. As per Microsoft One API guidelines - https://github.com/Microsoft/api-guidelines/blob/vNext/Guidelines.md#7102-error-condition-responses.

Name	Type	Description
code	
string

One of a server-defined set of error codes.

innererror	
Azure.Core.Foundations.InnerError

Inner error.

TextBlocklistMatch
The result of blocklist match.

Name	Type	Description
blocklistItemId	
string

The ID of the matched item.

blocklistItemText	
string

The content of the matched item.

blocklistName	
string

The name of the matched blocklist.

TextCategoriesAnalysis
Text analysis result.

Name	Type	Description
category	
TextCategory

The text analysis category.

severity	
integer

The value increases with the severity of the input content. The value of this field is determined by the output type specified in the request. The output type could be ‘FourSeverityLevels’ or ‘EightSeverity Levels’, and the output value can be 0, 2, 4, 6 or 0, 1, 2, 3, 4, 5, 6, or 7.

TextCategory
Text analyze category.

Name	Type	Description
Hate	
string

SelfHarm	
string

Sexual	
string

Violence	
string

'''
result = call_o1_model(prompt)
print(result)


In [ ]:
# Test building a plan using provided tools/functions - Minecraft plan scenario

prompt = '''You are driving a Minecraft character.  Your goal is to obtain a diamond sword.  Assume that you just started the game, and the only thing in your inventory is a crafting table.  
You have the following tools at your disposal:
+ wander(target) – walk around until you find the specified target.  The target can be a mob or an entity like a type of tree.
+ punch(target) – hit the target with your bare hands.  The target can be a mob or an entity like a type of tree.
+ mine – uses a pickaxe from the inventory to mine the block in front of you.  Fails if no pickaxe in your inventory.  
+ craft – use a crafting table to craft with existing items in your inventory.  The output goes into your inventory.  
+ place(item) – place something in your inventory
Build a plan using the tools to get a diamond sword.'''
result = call_o1_model(prompt)
print(result)

In [ ]:
# Productivity scenario - finding schedule conflicts

# I grabbed the U of M lineup from https://mgoblue.com/sports/football/schedule.  There are several options but I chose to download in Excel format which saves a .csv.  
# Then I saved my Outlook calendar with a custom date range of 9/12/2024 through 12/14/2024, with the “Limited details” level of detail so I had titles of events that may conflict; this saves an .ics.  
# To insert your own calendar, follow the instructions at https://answers.microsoft.com/en-us/outlook_com/forum/all/how-to-export-an-ics-file-so-receipent-can/ef0426e9-0816-4fc8-97a6-dd59f1b9b327 and then paste the contents of the resulting .ics file in the prompt below.

prompt = '''
Check the University of Michigan football schedule against my calendar and let me know if there are any conflicts.  
# University of Michigan football schedule:
Event	Start Date	Start Time	End Date	End Time	Location	Category	Description	Facility
Football vs Fresno State	8/31/2024	7:30PM	8/31/2024	10:30?PM	Ann Arbor, Mich.	Football	Football vs Fresno State Ann Arbor, Mich.	
Football vs Texas	9/7/2024	12:00PM	9/7/2024	3:00?PM	Ann Arbor, Mich.	Football	Football vs Texas Ann Arbor, Mich.	
Football vs Arkansas State	9/14/2024	12:00PM	9/14/2024	3:00?PM	Ann Arbor, Mich.	Football	Football vs Arkansas State Ann Arbor, Mich.	
Football vs USC	9/21/2024	3:30PM	9/21/2024	6:30?PM	Ann Arbor, Mich.	Football	Football vs USC Ann Arbor, Mich.	
Football vs Minnesota	9/28/2024	TBA	9/28/2024	3:00?AM	Ann Arbor, Mich.	Football	Football vs Minnesota Ann Arbor, Mich.	
Football at Washington	10/5/2024	TBA	10/5/2024	3:00?AM	Seattle, Wash.	Football	Football at Washington Seattle, Wash.	
Football at Illinois	10/19/2024	TBA	10/19/2024	3:00?AM	Champaign, Ill.	Football	Football at Illinois Champaign, Ill.	
Football vs Michigan State	10/26/2024	TBA	10/26/2024	3:00?AM	Ann Arbor, Mich.	Football	Football vs Michigan State Ann Arbor, Mich.	
Football vs Oregon	11/2/2024	TBA	11/2/2024	3:00?AM	Ann Arbor, Mich.	Football	Football vs Oregon Ann Arbor, Mich.	
Football at Indiana	11/9/2024	TBA	11/9/2024	3:00?AM	Bloomington, Ind.	Football	Football at Indiana Bloomington, Ind.	
Football vs Northwestern	11/23/2024	TBA	11/23/2024	3:00?AM	Ann Arbor, Mich.	Football	Football vs Northwestern Ann Arbor, Mich.	
Football at Ohio State	11/30/2024	12:00PM	11/30/2024	3:00?PM	Columbus, Ohio	Football	Football at Ohio State Columbus, Ohio	
Football vs Big Ten Championship Game	12/7/2024	8:00PM	12/7/2024	11:00?PM	Indianapolis, Ind.	Football	Football vs Big Ten Championship Game Indianapolis, Ind.	

__________________
# My calendar in ics format:
# TODO: insert your own ics here!  See https://answers.microsoft.com/en-us/outlook_com/forum/all/how-to-export-an-ics-file-so-receipent-can/ef0426e9-0816-4fc8-97a6-dd59f1b9b327 for how to do it.  

'''
result = call_o1_model(prompt)
print(result)